In [1]:
!pip install -U sagemaker

In [1]:
import sys

import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()

%store -r bucket_name
%store -r role
%store -r image_uri
%store -r model_data

# Define S3 paths for data and model artifacts
train_data = f's3://{bucket_name}/processed/housing/train.csv'
validation_data = f's3://{bucket_name}/processed/housing/validation.csv'
test_data = f's3://{bucket_name}/processed/housing/test.csv'
model_output = f's3://{bucket_name}/models/'


default_bucket = bucket_name
model_package_group_name = f"HousingModelPackageGroupName"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
!mkdir -p data

In [3]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

train_data = ParameterString(
    name="TrainData",
    default_value=train_data,
)
validation_data = ParameterString(
    name="ValidationData",
    default_value=validation_data,
)
test_data = ParameterString(
    name="TestData",
    default_value=test_data,
)
mse_threshold = ParameterFloat(name="MseThreshold", default_value=700.0)

In [4]:
!mkdir -p code

In [5]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-abalone-process",
    role=role,
    sagemaker_session=pipeline_session,
)

In [6]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

model_path = f"s3://{default_bucket}/HousingTrain"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
    sagemaker_session=pipeline_session,
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
)

train_args = xgb_train.fit(
    inputs={
        "train": TrainingInput(s3_data=train_data, content_type="csv"),
        "validation": TrainingInput(s3_data=validation_data, content_type="csv"),
    }
)

/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [7]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="HousingTrain",
    step_args=train_args,
)

In [8]:
%%writefile code/evaluation.py
import json
import pathlib
import pickle
import tarfile

import joblib
import numpy as np
import pandas as pd
import xgboost

from sklearn.metrics import mean_squared_error


if __name__ == "__main__":
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")

    model = pickle.load(open("xgboost-model", "rb"))

    test_path = "/opt/ml/processing/test/test.csv"
    df = pd.read_csv(test_path, header=None)

    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)

    X_test = xgboost.DMatrix(df.values)

    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    std = np.std(y_test - predictions)
    report_dict = {
        "regression_metrics": {
            "mse": {"value": mse, "standard_deviation": std},
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting code/evaluation.py


Next, create an instance of a `ScriptProcessor` processor and use it in the `ProcessingStep`.

In [9]:
from sagemaker.processing import ScriptProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep


script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="script-housing-eval",
    role=role,
    sagemaker_session=pipeline_session,
)

eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=test_data,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="code/evaluation.py",
)

In [10]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)
step_eval = ProcessingStep(
    name="HousingEval",
    step_args=eval_args,
    property_files=[evaluation_report],
)

In [11]:
from sagemaker.model import Model

model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

Define the `ModelStep` by providing the return values from `model.create()` as the step arguments.

In [12]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep

step_create_model = ModelStep(
    name="HousingCreateModel",
    step_args=model.create(instance_type="ml.m5.large", accelerator_type="ml.eia1.medium"),
)

In [13]:
from sagemaker.transformer import Transformer


transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=f"s3://{default_bucket}/HousingTransform",
)

Pass in the transformer instance and the `TransformInput` with the `batch_data` pipeline parameter defined earlier.

In [15]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)
step_register = ModelStep(name="HousingRegisterModel", step_args=register_args)

In [16]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join

step_fail = FailStep(
    name="HousingMSEFail",
    error_message=Join(on=" ", values=["Execution failed due to MSE >", mse_threshold]),
)

In [17]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value",
    ),
    right=mse_threshold,
)

step_cond = ConditionStep(
    name="HousingMSECond",
    conditions=[cond_lte],
    if_steps=[step_register, step_create_model],
    else_steps=[step_fail],
)

In [18]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"HousingPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        train_data,
        validation_data,
        test_data,
        mse_threshold,
    ],
    steps=[step_train, step_eval, step_cond],
)

### (Optional) Examining the pipeline definition

The JSON of the pipeline definition can be examined to confirm the pipeline is well-defined and the parameters and step properties resolve correctly.

In [19]:
import json


definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'TrainData',
   'Type': 'String',
   'DefaultValue': 's3://housing-dataset-5435v3/processed/housing/train.csv'},
  {'Name': 'ValidationData',
   'Type': 'String',
   'DefaultValue': 's3://housing-dataset-5435v3/processed/housing/validation.csv'},
  {'Name': 'TestData',
   'Type': 'String',
   'DefaultValue': 's3://housing-dataset-5435v3/processed/housing/test.csv'},
  {'Name': 'MseThreshold', 'Type': 'Float', 'DefaultValue': 700.0}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'HousingTrain',
   'Type': 'Training',
   'Arguments': {'A

In [20]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:952054755114:pipeline/HousingPipeline',
 'ResponseMetadata': {'RequestId': '39729372-d989-404d-bf60-9f6bf9e7d2bc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '39729372-d989-404d-bf60-9f6bf9e7d2bc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '83',
   'date': 'Sat, 22 Jun 2024 04:51:45 GMT'},
  'RetryAttempts': 0}}

Start the pipeline and accept all the default parameters.

In [21]:
execution = pipeline.start()

In [22]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:952054755114:pipeline/HousingPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:952054755114:pipeline/HousingPipeline/execution/b2yp6o9lb968',
 'PipelineExecutionDisplayName': 'execution-1719031908949',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'housingpipeline',
  'TrialName': 'b2yp6o9lb968'},
 'CreationTime': datetime.datetime(2024, 6, 22, 4, 51, 48, 884000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 6, 22, 4, 51, 48, 884000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:952054755114:user-profile/d-lrdmbhhktifc/wacuna',
  'UserProfileName': 'wacuna',
  'DomainId': 'd-lrdmbhhktifc',
  'IamIdentity': {'Arn': 'arn:aws:sts::952054755114:assumed-role/LabRole/SageMaker',
   'PrincipalId': 'AROA53KXCV4VD4SFVT6WB:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:952054755114:user-profile/d-lrdmbhhktifc

Wait for the execution to complete.

In [23]:
execution.wait()

List the steps in the execution. These are the steps in the pipeline that have been resolved by the step executor service.

In [24]:
execution.list_steps()

[{'StepName': 'HousingCreateModel-CreateModel',
  'StartTime': datetime.datetime(2024, 6, 22, 4, 56, 46, 749000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 6, 22, 4, 56, 47, 791000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:952054755114:model/pipelines-b2yp6o9lb968-HousingCreateModel-C-jbeZqFoY4P'}},
  'AttemptCount': 1},
 {'StepName': 'HousingRegisterModel-RegisterModel',
  'StartTime': datetime.datetime(2024, 6, 22, 4, 56, 46, 749000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 6, 22, 4, 56, 47, 577000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:952054755114:model-package/HousingModelPackageGroupName/1'}},
  'AttemptCount': 1},
 {'StepName': 'HousingMSECond',
  'StartTime': datetime.datetime(2024, 6, 22, 4, 56, 46, 270000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 6, 22, 4, 56, 46, 435000, tzinfo=tzlocal())

### Examining the Evaluation

Examine the resulting model evaluation after the pipeline completes. Download the resulting `evaluation.json` file from S3 and print the report.

In [25]:
from pprint import pprint


evaluation_json = sagemaker.s3.S3Downloader.read_file(
    "{}/evaluation.json".format(
        step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
    )
)
pprint(json.loads(evaluation_json))

{'regression_metrics': {'mse': {'standard_deviation': 25.252699087316618,
                                'value': 639.7426725514896}}}


### Lineage

Review the lineage of the artifacts generated by the pipeline.

In [26]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

{'StepName': 'HousingTrain', 'StartTime': datetime.datetime(2024, 6, 22, 4, 51, 50, 239000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 6, 22, 4, 54, 12, 41000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:952054755114:training-job/pipelines-b2yp6o9lb968-HousingTrain-W2TS175u6R'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...-5435v3/processed/housing/validation.csv,Input,DataSet,ContributedTo,artifact
1,s3://...taset-5435v3/processed/housing/train.csv,Input,DataSet,ContributedTo,artifact
2,68331...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...singTrain-W2TS175u6R/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'HousingEval', 'StartTime': datetime.datetime(2024, 6, 22, 4, 54, 12, 818000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 6, 22, 4, 56, 45, 550000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:952054755114:processing-job/pipelines-b2yp6o9lb968-HousingEval-IqFitwBO84'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...1efdc9e2d552feb7e97dadd942/evaluation.py,Input,DataSet,ContributedTo,artifact
1,s3://...ataset-5435v3/processed/housing/test.csv,Input,DataSet,ContributedTo,artifact
2,s3://...singTrain-W2TS175u6R/output/model.tar.gz,Input,Model,ContributedTo,artifact
3,68331...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
4,s3://...024-06-22-04-51-27-332/output/evaluation,Output,DataSet,Produced,artifact


{'StepName': 'HousingMSECond', 'StartTime': datetime.datetime(2024, 6, 22, 4, 56, 46, 270000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 6, 22, 4, 56, 46, 435000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'Condition': {'Outcome': 'True'}}, 'AttemptCount': 1}


None

{'StepName': 'HousingRegisterModel-RegisterModel', 'StartTime': datetime.datetime(2024, 6, 22, 4, 56, 46, 749000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 6, 22, 4, 56, 47, 577000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:952054755114:model-package/HousingModelPackageGroupName/1'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...singTrain-W2TS175u6R/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,68331...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
2,HousingModelPackageGroupName-1-PendingManualAp...,Input,Approval,ContributedTo,action
3,HousingModelPackageGroupName-1719032207-aws-mo...,Output,ModelGroup,AssociatedWith,context


{'StepName': 'HousingCreateModel-CreateModel', 'StartTime': datetime.datetime(2024, 6, 22, 4, 56, 46, 749000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 6, 22, 4, 56, 47, 791000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:952054755114:model/pipelines-b2yp6o9lb968-HousingCreateModel-C-jbeZqFoY4P'}}, 'AttemptCount': 1}


None

In [27]:
execution = pipeline.start(
    parameters=dict(
        ModelApprovalStatus="Approved",
    )
)

In [28]:
execution.wait()

In [29]:
execution.list_steps()

[{'StepName': 'HousingCreateModel-CreateModel',
  'StartTime': datetime.datetime(2024, 6, 22, 5, 6, 35, 274000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 6, 22, 5, 6, 36, 623000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:952054755114:model/pipelines-ft4ejt8g9lqp-HousingCreateModel-C-3c2Ah3SGOa'}},
  'AttemptCount': 1},
 {'StepName': 'HousingRegisterModel-RegisterModel',
  'StartTime': datetime.datetime(2024, 6, 22, 5, 6, 35, 274000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 6, 22, 5, 6, 36, 622000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:952054755114:model-package/HousingModelPackageGroupName/2'}},
  'AttemptCount': 1},
 {'StepName': 'HousingMSECond',
  'StartTime': datetime.datetime(2024, 6, 22, 5, 6, 34, 212000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 6, 22, 5, 6, 34, 637000, tzinfo=tzlocal()),
  'S

Apart from that, you might also want to adjust the MSE threshold to a smaller value and raise the bar for the accuracy of the registered model. In this case you can override the MSE threshold like the following:

In [30]:
execution = pipeline.start(parameters=dict(MseThreshold=10.0))

If the MSE threshold is not satisfied, the pipeline execution enters the `FailStep` and is marked as failed.

In [31]:
try:
    execution.wait()
except Exception as error:
    print(error)

Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state: For expression "PipelineExecutionStatus" we matched expected path: "Failed"


In [32]:
execution.list_steps()

[{'StepName': 'HousingMSEFail',
  'StartTime': datetime.datetime(2024, 6, 22, 5, 11, 46, 467000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 6, 22, 5, 11, 47, 142000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'FailureReason': 'Execution failed due to MSE > 10.0',
  'Metadata': {'Fail': {'ErrorMessage': 'Execution failed due to MSE > 10.0'}},
  'AttemptCount': 1},
 {'StepName': 'HousingMSECond',
  'StartTime': datetime.datetime(2024, 6, 22, 5, 11, 45, 420000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 6, 22, 5, 11, 45, 853000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Condition': {'Outcome': 'False'}},
  'AttemptCount': 1},
 {'StepName': 'HousingEval',
  'StartTime': datetime.datetime(2024, 6, 22, 5, 9, 12, 211000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 6, 22, 5, 11, 44, 894000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:952054755114:processing-j

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker-pipelines|tabular|abalone_build_train_deploy|sagemaker-pipelines-preprocess-train-evaluate-batch-transform.ipynb)
